Partial Least Squares - Discriminant Analysis (PLS-DA)
===

Author: Nathan A. Mahynski

Date: 2023/09/12

Description: Discussion and examples of different PLS-DA approaches.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mahynski/pychemauth/blob/main/docs/jupyter/gallery/plsda.ipynb)

[PLS](pls.ipynb) can also be applied to classification problems.  The general idea is to perform a PLS(2) decomposition
between $X$ and $\vec{y}$, where now $\vec{y}$ is one-hot encoded for the different classes.  For binary classification, a simple 0 or 1 is adequate.  The scores that come from the PLS decomposition are then used as an input to a
classification model.  So really, PLS-DA is just using PLS to find a good subspace, then performing classication
on the transformed coordinates in that space; this classification is the "discrimination" which can be done
by any number of methods.  The PLS outputs floating point numbers not integers, so a decision needs to be made
on how to "cluster" and assign points to a given class. Some common methods are:

1. Closest class centroid - obviously that Euclidean distance in PLS-DA score space (projection) is not necessarily a good representation of class differences so you need to validate this with test set, etc. See this [paper](https://link.springer.com/article/10.1007/s11306-007-0099-6) and more discussion in [Pomerantsev et al.](https://onlinelibrary.wiley.com/doi/abs/10.1002/cem.3030).

2. Thresholding (some cutoff distance).

3. Some statistical confidence bounds based on tolerable mis-classfication error rates.

4. Build logistic regression, or other decision boundary, etc. in score (projected) space - this is better than option (1) which avoids relying on distance being proportional to likelihood of a class.

This essentially just uses (supervised) PLS to find a good subspace to project into.  However, we could also use something like LDA instead (discussed in the previous section); for LDA we are bounded by `#dimensions <= min(n_classes - 1, n_features)`, which we are not in PLS.  So if `n_classes` is low, LDA can only find a very low-D subspace; it may be better to find some space between that and `n_features`, which PLS can provide.

PLS-DA is widely applied in cheminformatics research since we often have a $p > n$ instance which can be handled
automatically with PLS.  This is especially true of -omics fields.   However, nonlinear techniques such as [artificial neural networks can also be used](https://link.springer.com/article/10.1007/s11306-020-1640-0) have been shown to perform as well. A thorough review of the state of the art PLS-DA by Lee et al. is available [here](https://pubs.rsc.org/en/content/articlehtml/2018/an/c8an00599k)

In [5]:
if 'google.colab' in str(get_ipython()):
    !pip install git+https://github.com/mahynski/pychemauth@main
    import os
    os.kill(os.getpid(), 9) # Automatically restart the runtime to reload libraries

In [6]:
try:
    import pychemauth
except:
    raise ImportError("pychemauth not installed")

import matplotlib.pyplot as plt
%matplotlib inline

import watermark
%load_ext watermark

%load_ext autoreload
%autoreload 2

In [114]:
import sklearn

import numpy as np

from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.datasets import make_blobs

from pychemauth.classifier.plsda import PLSDA
from pychemauth.preprocessing.scaling import CorrectedScaler

In [2]:
%watermark -t -m -v --iversions

UsageError: Line magic function `%watermark` not found.


Starting Out
---

["Multiclass partial least squares discriminant analysis: Taking the right way—A critical tutorial," by
Pomerantsev and Rodionova, Journal of Chemometrics, 32 (2018)](https://analyticalsciencejournals.onlinelibrary.wiley.com/doi/abs/10.1002/cem.3030) suggests 2 approaches to PLS-DA. These are "hard" and "soft" PLS-DA, which are distinguished by how they determine their discrimination boundaries.  Both begin in the same way.  First, let's consider a naive approach to PLS-DA.

1. One-hot encode $Y$ for different classes.

> This means that classes form the vertices of $k$-dimensional simplex, where we have $k$ classes.  Following [Indahl et al.](https://onlinelibrary.wiley.com/doi/abs/10.1002/cem.1061), you could reduce the dummy matrix to remove a dimension;  if you have 4 classes, this forms a tetrahedron - i.e., represent classes as +1 for k-1 instances and for the last one indicate it as all zeros.
>
> Convert
>
> $$
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{bmatrix}
$$
>
> to
>
>$$
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 \\
0 & 0 & 0 \\
\end{bmatrix}
$$
>
>The first matrix is connected by a hyperplane and so strictly the space spanned by these classes is $k$-1 dimensional.  This is made more explicitly clear by the second matrix which clearly has rank $k-1$ and encloses a tetrahedral volume.

<img src="../../_static/fig1_pomerantsev_2018.png" style="width:500px;">

2. Center $X$ (and possibly scale, too), and mean-center $Y$.

3. Perform [PLS2 regression](https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.PLSRegression.html) to obtain the predictions, $\hat{Y}$.

4. Compare $\hat{Y}$ to the OHE vectors and choose the closest one to assign a class.  

However, step 4 is tricky because of the fact that $\hat{Y}$ has rank $k$-1.  if you tried to compute a Euclidean distance ($k$-D space) between one vertex and a given prediction, it would be possible - but this would not really be correct since we really need to the distance in the ($k$-1)-D space. Moreover, if you tried to compute a Mahalanobis distance this would fail because the covariance matrix will be singular. If you just use Indahl's approach and do PLS2 with the second matrix, it seems you can proceed; however, Pomerantsev and Rodionova proposed that you instead use the normal OHE matrix (the first one above) and then perform PCA on the $\hat{Y}$ response.  The PCA will naturally only have $k-1$ non-zero eigenvalues, so you can simply use this to project the responses into that (hyper)plane.  You can then compute the distances in that hyperplane to perform classification.  These steps are as follows:

1. One-hot encode response vector, $Y_{\rm train}$.

2. Perform PLS2 to get prediction, $\hat{Y}_{\rm train}$, after appropriate centering/scaling.

3. Perform PCA on $\hat{Y}_{\rm train}$ - to do this, you must first center $\hat{Y}_{\rm train}$. Project into $k-1$ dimensional space to obtain the PCA loadings matrix $T$.

4. Classify points in $T$.

In summary, $(X, Y) \xrightarrow{\text{PLS2}} \hat{Y} \xrightarrow{\text{PCA}} T$.  This is essentially a composition of multiple subspace projection methods, but ultimately only the PCA scores (T) are used to do the classification.

This assumes that $k < n_{\rm features}$.  Since we are projecting into a $k$-D space and relying on a PCA transformation of that space this is a requirement.  Usually when PLS is being used this is the case.

Pomerantsev and Rodionova propose 2 different ways to perform step 4: a hard classification and a soft classification.

Hard PLS-DA
---

**Hard classification assigns an object to 1 and only 1 class.**  It is not generally valid to use this approach for authentication studies since this method will not be applicable in "open world" conditions where a classifer may encounter classes different from those trained on. Still, this conventional "closed world" classifier can still be powerful. See [Rodionova et al.](https://www.sciencedirect.com/science/article/pii/S0165993615302193) for more details.

Recall that [LDA](lda.ipynb) uses multivariate Gaussians to model class probabilities; this essentially assigns a point to the class by finding the smallest Mahalanobis distance to a known class center.  We can use a similarly inspired approach here.

The pooled covariance matrix is known from PCA:

$$
cov(T^T) = \Lambda \sim T^TT 
$$

It is a diagonal matrix because PCA found orthogonal components $\left( \Lambda = {\rm diag}\left( \lambda_1, \dots, \lambda_{k-1} \right) \right)$.  The class centers, $c_k$, are computed by column centering them (since this was done in the PCA step to projection $\hat{Y}$) then using the PCA eigenvectors to project to this $k-1$ dimensional space.

The distance from a projected point, $t_i$, to a class, $k$ is given by its Mahalanobis distance:

$$
d_{ik} = (t_i - c_k)\Lambda^{-1}(t_i - c_k)^{T}
$$

It is perfectly fine to use the Mahalanobis distance as classification metric, but it is not identical to the [LDA](lda.ipynb) decision function, strictly speaking.  You could also use other classifications approaches like trees, QDA, etc.

In [153]:
from scipy.spatial.distance import mahalanobis

def hard_plsda(X_test, X_train, y_train, n_components=2):
    # 1. Preprocess data (one hot encoding, centering)
    ohencoder = OneHotEncoder(sparse_output=False) # Convert integers to OHE
    x_pls_scaler = CorrectedScaler(with_mean=True, with_std=True) # Center and (optionally) scale X
    y_pls_scaler = CorrectedScaler(with_mean=True, with_std=False) # Center do not scale Y

    y_train = y_pls_scaler.fit_transform(ohencoder.fit_transform(y_train.reshape(-1,1)))
    X_train = x_pls_scaler.fit_transform(X_train)

    X_test = x_pls_scaler.transform(X_test)

    # 2. PLS2 - predict y_hat
    plsda = PLSRegression(n_components=n_components,
                          max_iter=5000,
                          tol=1.0e-9,
                          scale=False) # Already scaled, centered as needed
    _ = plsda.fit(X_train, y_train)
    
    # Y was centered for PLS fitting, so de-center to return to "normal"
    y_hat_train = y_pls_scaler.inverse_transform(plsda.predict(X_train))
    y_hat_test = y_pls_scaler.inverse_transform(plsda.predict(X_test))

    # 3. Perform PCA on y_hat_train - use k-1 dimensions
    # Note that sklearn's PCA automatically centers the input so we do not need to do it explicitly
    pca = PCA(n_components=len(ohencoder.categories_[0])-1, random_state=0)
    T_train = pca.fit_transform(y_hat_train) # The means of y_hat_train are stored
    T_test = pca.transform(y_hat_test) # The means are re-used here to center
    class_centers = pca.transform(np.eye(len(ohencoder.categories_[0]))) # Centers based on y_hat_train automatically

    # As a sanity check, proof that covariance of T is diagonal matrix made of eigenvalues from PCA transform
    L = np.cov(T_train.T)
    assert(np.allclose(np.cov(T_train.T), np.eye(len(pca.explained_variance_))*pca.explained_variance_))

    # 4. Compute Mahalanobis distance (squared) and classify based on the smallest distance (squared)
    d2 = []
    for t in T_test:
        d2.append([
            np.matmul(
                np.matmul(
                    (t - class_centers[i]),
                    np.linalg.inv(L)),
                (t - class_centers[i]).reshape(-1,1)
            )[0] for i in range(len(class_centers))
        ])
    d2 = np.array(d2)
    
    prediction = np.array([ohencoder.categories_[0][c] for c in np.argmin(d2, axis=1)] )

    return prediction, d2, T_train, T_test

In [164]:
X, Y = sklearn.datasets.make_blobs(
    n_samples=100,
    n_features=5,
    centers=3,
    cluster_std=1,
    shuffle=True,
    random_state=0
)

X_train, y_train = X[:80], Y[:80]
X_test, y_test = X[80:], Y[80:]

prediction, squared_distances, T_train, T_test = hard_plsda(X_test, X_train, y_train, n_components=2)

In [181]:
# In this case, the PLS-DA predicts the target classes perfectly on the test set.
print('Prediction\tActual\tEqual?\n------------------------------')
for pred, act in zip(prediction, y_test):
    print(pred, '\t\t', act, '\t', pred==act)

Prediction	Actual	Equal?
------------------------------
0 		 0 	 True
0 		 0 	 True
1 		 1 	 True
0 		 0 	 True
0 		 0 	 True
0 		 0 	 True
1 		 1 	 True
1 		 1 	 True
2 		 2 	 True
2 		 2 	 True
0 		 0 	 True
1 		 1 	 True
0 		 0 	 True
2 		 2 	 True
0 		 0 	 True
0 		 0 	 True
2 		 2 	 True
1 		 1 	 True
2 		 2 	 True
2 		 2 	 True


In [175]:
.# The same calculation can be accomplished with pychemauth
plsda = PLSDA(n_components=2, style='hard', scale_x=True)
_ = plsda.fit(X_train, y_train)

In [176]:
assert np.all(plsda.predict(X_test) == prediction)
assert np.allclose(plsda.transform(X_train), T_train)
assert np.allclose(plsda.transform(X_test), T_test)
assert np.allclose(plsda.mahalanobis(X_test), squared_distances)

Soft PLS-DA
---

Rather than using an LDA-like distance (which assumes all classes have the same covariance matrix), you can instead use a QDA-like distance which uses a separate within-class covariance matrix for each class.  Each within-class covariance matrix is given by:

$$
S_k = \frac{1}{I_k - 1} \sum_{i \in \omega(k)} (t_i-c_k)^T(t_i-c_k)
$$

where there are $I_k$ samples from class $k$ and $\omega(k)$ denotes which data points (in the training set) belong to class $k$.  The Mahalanobis distance (squared) as:

$$
d_{ik} = (t_i - c_k)S_k^{-1}(t_i - c_k)^{T}
$$

Assuming the distances follow a chi-squared distribution we can establish a critical distance below which we assume point i belongs to class $k$, and beyond which we assume it does not.

$$
d_{\rm crit} = \chi^{-2}(1-\alpha, k-1)
$$

where $\chi^{-2}$ is the quantile of the chi-squared distribution with $k$-1 degrees of freedom and a type I error rate of $\alpha$. $H_0$ is that a point belongs to class $k$; type I error is the rejection of a true $H_0$ (that it belongs to class $k$) so this represents the rate at which we reject that a point belongs to a class that it truly does belong to.  

One can also specify an outlier threshold:

$$
d_{\rm out} = \chi^{-2}((1-\gamma)^{1/I_k}, k-1)
$$

We expect the [chi-squared distribution](https://en.wikipedia.org/wiki/Chi-square_distribution) since it represents the sum of squres of random variates; here, each direction is assumed to be a random variable and thus, computing a Mahalanobis distance (in a $k$-1 dimensional space) as done above results in a sum of squares.

**Thus, Soft PLS-DA may assign a point to 1 or more classes, or to none at all.  This means it can be used for authentication studies.**